# Purpose

The purpose of this script is to analyze stock following principals outlines by Benjamin Graham. These principles involve the following guidlines

- PE ratio below 9 but positivie
- Current Ratio (Current Assets/Current Liab) greater than 1.5
- Debt Load (Total Debt/ Current Assets) Less than or equal to 1.1
- Positive EPS for each quarter in the last year
- Price To Book ratio less than or equal to 1.2 
- Look for dividends

Additionally I will be calculating the following variables for my own interest
- NCAV (Net Current Assets Value) = (Net Assets-TotalLiabilities-preferred stock)

## Load Name of Stocks
The section uses the yahoo finance library to get the names of many stocks. 

In [1]:
import yahoo_fin.stock_info as si
dow_list = si.tickers_dow() #Loads Tickers in dow jones
nasdaq_list = si.tickers_nasdaq() #Loads Tickers in NASDAQ
sp500_list = si.tickers_sp500()
other_list = si.tickers_other() #Loads Tickers in Other Indexes


## Load yfinance and Create Functions To Read Data I am interested In 
The yfinance library is what will primarily be used to load information for each of the stock tickers. Within this libary 3 primary functions will be called they are   

- stk.info #(used for PE, EPS, Price to Book values)
- stk.balance_sheet (used for Current Ratio, Debt Load, NCAV)
- stk.dividends (used for dividend calculations

In order to use these functions more efficiently they will be integrated into a number of my own creation. The first of these functions is below and is used to read useful information from the stk.info call

In [2]:
import yfinance as yf
import numpy
import pandas as pd

def extractInfo(Ticker):
    commonStock = False
    divRate = False
    forPE = 0
    trailPE = 0
    forEPS = 0
    trailEPS = 0
    P2B = 0
    
    #Ticers with these values in their name were often for funds preferred stock etc
    #So this line removes those other investment mediums
    if '$' in Ticker or '.' in Ticker:
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B
    
    stk = yf.Ticker(Ticker) #Creates object to load stock data
    stkInfo = stk.info #Loads basic stock info 
    
    #In yfinance all common stock have forwardPE listed but funds and other investments do not.
    #So this line removes those other investment mediums
    if 'forwardPE' not in stkInfo or stkInfo['forwardPE'] == None: 
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B
    
    commonStock = True
    
    #Reads stock info where it is available to be read
    divRate = stkInfo['dividendRate'] if 'dividendRate' in stkInfo else 'None'
    forPE = stkInfo['forwardPE']
    trailPE = stkInfo['trailingPE'] if 'trailingPE' in stkInfo else 'Negative'
    forEPS = stkInfo['forwardEps'] if 'forwardEps' in stkInfo else 'None'
    trailEPS = stkInfo['trailingEps'] if 'trailingEps' in stkInfo else 'None'
    P2B = stkInfo['priceToBook'] if 'priceToBook' in stkInfo else 'None'
    
    return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk

The second function will be responsible for reading the balance sheet for the stk. It is assumed here that the function extractInfo has already been read ahead of this function and has prevented any non-common-stock investments from progressing further into the data extraction process 

In [3]:
def extractBalSheet(stk):
    stkBal= stk.balancesheet #Returns last four years of annual balance sheet
    recBal = stkBal[stkBal.columns[0]] #Returns the most recent balance sheet
    
    curAsts = recBal['Total Current Assets'] if 'Total Current Assets' in recBal else 'None'
    curLiabs = recBal['Total Current Liabilities'] if 'Total Current Liabilities' in recBal else 'None'
    totAsts = recBal['Total Assets'] if 'Total Assets' in recBal else 'None'
    totLiabs = recBal['Total Liab'] if 'Total Liab' in recBal else 'None'
    
    stkHoldEqty = totAsts-totLiabs if (type(totAsts)!=str and type(totLiabs)!=str) else 'None'
    curRat = curAsts/curLiabs if (type(curAsts)!=str and type(curLiabs)!=str and curLiabs!=0) else 'None'
    debtLoad = totLiabs/curAsts if (type(curAsts)!=str and type(totLiabs)!=str and curAsts!=0) else 'None'
    
    return stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs
    

The third function is responsible for reading the earnings of the data for the last four monthes.

In [4]:
def extractPrev4QuartEarnings(stk):
    stkQuartEarn = stk.quarterly_earnings
    earn = stkQuartEarn['Earnings'] if 'Earnings' in stkQuartEarn.columns else 'None'
    numQ = earn.shape[0]
    mostRec = earn[-1] if numQ >= 1 else 'None' #Most recent is last value
    secMostRec = earn[-2] if numQ >= 2 else 'None'
    thrdMostRec = earn[-3] if numQ >= 3 else 'None'
    frthMostRec = earn[-4] if numQ >= 4 else 'None'
    
    return mostRec, secMostRec, thrdMostRec, frthMostRec
    

## Loop Through Stocks in the Dow

In [27]:
stkDFwFor = pd.DataFrame(columns = ['Ticker', 'divRate','forward PE','Trail PE','Forward EPS','Trail EPS','Price to Book',
                            'stock Holder Equity','Current Ration','Debt Load','Current Assets',
                            'Current Liabillities','Total Assets','Total Liabilities',
                            'Most Recent Quarter Earnings','2nd MRE','3rd MRE','4th MRE'])
stkDF = pd.DataFrame(columns = ['Ticker', 'divRate','Trail PE','Trail EPS','Price to Book',
                            'stock Holder Equity','Current Ratio','Debt Load','Current Assets',
                            'Current Liabillities','Total Assets','Total Liabilities',
                            'Most Recent Quarter Earnings','2nd MRE','3rd MRE','4th MRE'])

for Ticker in dow_list:
    commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk = extractInfo(Ticker)
    
    if commonStock == True:
        stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs = extractBalSheet(stk)
        mostRec, secMostRec, thrdMostRec, frthMostRec = extractPrev4QuartEarnings(stk)
        
        nrwFor = [Ticker,divRate,forPE,trailPE,forEPS,trailEPS,P2B,
             stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs,
             mostRec, secMostRec, thrdMostRec, frthMostRec]
        
        nr = [Ticker,divRate,trailPE,trailEPS,P2B,
              stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs,
              mostRec, secMostRec, thrdMostRec, frthMostRec]
        
        stkDFwFor = stkDFwFor.append(pd.Series(nrwFor,
                                      index = stkDFwFor.columns),
                            ignore_index=True)
        stkDF = stkDF.append(pd.Series(nr,
                                      index = stkDF.columns),
                            ignore_index=True)
        
        
        

In [22]:
print(stkDF)
print(stkDF['Ticker'])
stkDF.index = stkDF['Ticker']
print(stkDF.loc['AAPL'])
data = stkDF.loc['AXP']
print(data['Trail PE'])


       Ticker divRate  Trail PE  Trail EPS Price to Book  stock Holder Equity  \
Ticker                                                                          
AAPL     AAPL    0.82   37.9207      3.280       32.3149         6.533900e+10   
AMGN     AMGN     6.4   18.4644     12.399         12.19         9.673000e+09   
AXP       AXP    1.72   30.4087      4.062       4.54686         2.307100e+10   
BA         BA    None  Negative     -7.887          None        -8.300000e+09   
CAT       CAT    4.12   29.6485      6.032       6.49925         1.462900e+10   
CRM       CRM    None   59.0617      3.858       5.16654         3.388500e+10   
CSCO     CSCO    1.44   17.9636      2.470       4.90927         3.792000e+10   
CVX       CVX    5.16  Negative     -6.182       1.29719         1.452080e+11   
DIS       DIS    None  Negative     -1.588       3.32886         9.751200e+10   
DOW       DOW     2.8  Negative     -3.167       3.27529         1.409400e+10   
GS         GS       5   13.7

## Filters Stocks Based on Benjamin Graham Requirements

Create a function to filter the stock

In [33]:
def filterStock(stkDF,maxPE,minCurRat,maxDebtLoad,maxPB):
    if type(data['Trail PE']) ==str or data['Trail PE'] > maxPE:
        return False

SyntaxError: unexpected EOF while parsing (<ipython-input-33-3791b2980471>, line 2)

In [30]:
maxPE = 20 #Graham Suggests 9
minCurRat = 1.5 #Graham Suggests 1.5
maxDebtLoad = 1.1 #Graham Suggests 1.1
maxPB = 1.2 #Graham Suggests 1.2

stkDF.index = stkDF['Ticker']
tkrList = stkDF.index

stkDfFltr = pd.DataFrame(columns = ['Ticker', 'divRate','Trail PE','Trail EPS','Price to Book',
                            'stock Holder Equity','Current Ratio','Debt Load','Current Assets',
                            'Current Liabillities','Total Assets','Total Liabilities',
                            'Most Recent Quarter Earnings','2nd MRE','3rd MRE','4th MRE'])

for tkr in tkrList:
    print(tkr)
    data = stkDF.loc[tkr]
    if type(data['Trail PE'])!=str and data['Trail PE'] <= maxPE:
        stkDfFltr = stkDfFltr.append(pd.Series(data,
                                      index = stkDfFltr.columns),
                            ignore_index=True)
    

AAPL
AMGN
AXP
BA
CAT
CRM
CSCO
CVX
DIS
DOW
GS
HD
HON
IBM
INTC
JNJ
JPM
KO
MCD
MMM
MRK
MSFT
NKE
PG
TRV
UNH
V
VZ
WBA
WMT


In [31]:
print(stkDfFltr)

  Ticker  divRate   Trail PE  Trail EPS  Price to Book  stock Holder Equity  \
0   AMGN     6.40  18.464392     12.399      12.189980         9.673000e+09   
1   CSCO     1.44  17.963562      2.470       4.909272         3.792000e+10   
2     GS     5.00  13.768374     17.347       1.052075         9.197800e+10   
3    IBM     6.52  14.247988      8.823       5.281711         2.098500e+10   
4   INTC     1.32   9.921706      5.109       2.786236         7.750400e+10   
5    JPM     3.60  15.908201      7.669       1.542761         2.613300e+11   
6    MRK     2.60  18.370142      4.528       7.209847         2.600100e+10   
7    TRV     3.40  15.414245      8.775       1.230251         2.594300e+10   
8    UNH     5.00  19.981617     17.409       5.060739         6.216200e+10   
9     VZ     2.51  13.902715      4.420       3.908039         6.283500e+10   

   Current Ratio  Debt Load  Current Assets  Current Liabillities  \
0       1.436697   2.713341    1.844000e+10          1.283500